In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore some warnings 
import warnings
warnings.filterwarnings('ignore')

# Some useful constants
SEED = 1
DATA_PATH = './data/'

In [2]:
import sqlite3

In [ ]:
conn = connect('Steam_db')

In [ ]:
conn = connect("stocks.db")
data = pd.read_csv("FB.csv")
data.to_sql("meta",conn)
df = pd.read_sql("SELECT * FROM meta LIMIT 5",conn)
print(df)

In [3]:
import gzip

f = gzip.open(DATA_PATH+'steam.sql.gz','rb')

In [5]:
#Connect to Database
conn = sqlite3.connect(DATA_PATH+'') 

In [ ]:
# Read the sql file
query = open('filename.sql', 'r')

# connection == the connection to your database, in your case prob_db
DF = pd.read_sql_query(query.read(), connection)
query.close() 

In [ ]:
subset_Steam_DB = pd.read_sql('SELECT * LIMIT 100000', conn)

In [ ]:
# from sqlite3 import connect
# conn = connect(':memory:')